In [1]:
##### creating shapefile out of gw site description file
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
fit_to_meter = 0.3048
# Path to the groundwater daily stations site locations
gw_daily_path = r'/data/MyDataBase/SWATGenXAppData/groundwater_daily_stations/site_locations'
# Read the CSV, skipping the first 30 rows and using tab as separator
site_locations = pd.read_csv(gw_daily_path, skiprows=27, sep='\t')
# Selecting only the required columns
site_locations = site_locations[['site_no', 'dec_lat_va', 'dec_long_va', 'dec_coord_datum_cd','alt_va']]
# Create a 'geometry' column with Points made from latitude and longitude
site_locations['geometry'] = site_locations.apply(lambda row: Point(row.dec_long_va, row.dec_lat_va), axis=1)
# Convert DataFrame to GeoDataFrame, setting the original coordinate reference system (CRS) to 'EPSG:4269' for NAD83
site_locations_gdf = gpd.GeoDataFrame(site_locations, geometry='geometry', crs='EPSG:4269')
# Transforming the GeoDataFrame to the new CRS 'EPSG:26990'
site_locations_gdf['alt_va_m'] = fit_to_meter*site_locations_gdf['alt_va']
site_locations_gdf['alt_va_f'] = site_locations_gdf['alt_va']
site_locations_gdf = site_locations_gdf.drop(columns='alt_va')
site_locations_gdf = site_locations_gdf.to_crs('EPSG:26990')

huc12 = pd.read_pickle(r'D:\MyDataBase\NHDPlusData\WBDHU12.pkl')
huc8 = pd.read_pickle(r'D:\MyDataBase\NHDPlusData\WBDHU8.pkl')
watersheds = pd.read_pickle(r'D:\MyDataBase\NHDPlusData\watersheds.pkl')

site_locations_gdf = site_locations_gdf.sjoin(watersheds[['NHDPlusID','geometry']]).drop(columns = 'index_right')
site_locations_gdf = site_locations_gdf.sjoin(huc8[['huc8','geometry']]).drop(columns = 'index_right')
site_locations_gdf = site_locations_gdf.sjoin(huc12[['huc12','geometry']]).drop(columns = 'index_right')

site_locations_gdf['huc12'] = site_locations_gdf['huc12'].astype('int64')
site_locations_gdf['huc8'] = site_locations_gdf['huc8'].astype('int64')
site_locations_gdf['NHDPlusID'] = site_locations_gdf['NHDPlusID'].astype('int64')

site_locations_gdf.to_file(r'/data/MyDataBase/SWATGenXAppData/groundwater_daily_stations/statons_location')

C:\Users\rafieiva\AppData\Local\Temp\2\ipykernel_95632\1584377393.py:34: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  site_locations_gdf.to_file(r'/data/MyDataBase/SWATGenXAppData/groundwater_daily_stations/statons_location')


In [2]:
from datetime import datetime
import pandas as pd

site_id = []
date = []
wl_f = []

with open('/data/MyDataBase/SWATGenXAppData/groundwater_daily_stations/gw_daily', 'r') as file:
    lines = file.readlines()
    for line in lines:
        if line.split()[0] == 'USGS':
            site_id.append(line.split('\t')[1])
            date.append(line.split('\t')[2])  # YYYY-MM-DD : 2002-06-18
            wl_f.append(line.split('\t')[3])  # Feet below land surface
    

gw_daily = pd.DataFrame({'site_id': site_id, 'date': date, 'wl_f': wl_f})
gw_daily['date'] = pd.to_datetime(gw_daily['date'])  # Convert 'date' to datetime
# Convert 'wl_f' to float, with non-convertible values becoming NaN
gw_daily['wl_f'] = pd.to_numeric(gw_daily['wl_f'], errors='coerce')
# Assuming 'fit_to_meter' is defined earlier in your code as the conversion factor from feet to meters
gw_daily['wl_m'] = fit_to_meter * gw_daily['wl_f']

In [4]:
from pandas import date_range
import numpy as np
# Create a date range from 1990-01-01 to 2022-12-31
date_range_all = date_range(start='1990-01-01', end='2022-12-31')

# Unique stations
stations = gw_daily.site_id.unique()

for station in stations:
    # Filter data for the current station
    station_data = gw_daily[gw_daily.site_id == station]
    
    # Filter data within the date range 1990 to 2022
    station_data = station_data[(station_data['date'].dt.year >= 1990) & (station_data['date'].dt.year <= 2022)]

    # Reindexing to fill in missing dates
    station_data.set_index('date', inplace=True)
    station_data = station_data.reindex(date_range_all, fill_value=np.nan)

    # Resetting the index to bring 'date' back as a column
    station_data.reset_index(inplace=True)
    station_data.rename(columns={'index': 'date'}, inplace=True)

    # Add back the station ID for the current station
    station_data['site_id'] = station

    # Export data
    export_path = f'/data/MyDataBase/SWATGenXAppData/groundwater_daily_stations/{station}.csv'
    try:
        alt_va_m = site_locations_gdf[site_locations_gdf.site_no==int(station_data.site_id.unique()[0])].alt_va_m.values[0]
        station_data ['head_m'] = alt_va_m - station_data ['wl_m']
        station_data ['head_m'] = station_data ['head_m'].astype(float).round(2)
        station_data ['wl_m'] = station_data ['wl_m'].astype(float).round(2)
        station_data.to_csv(export_path, index=False)
    except Exception as e:
        print(e)

index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0


In [6]:
import shutil
import os
BASE_PATH= '/data/MyDataBase/SWATGenXAppData/'
LEVEL = 'huc12'

NAMES = os.listdir(f'/data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/{LEVEL}')
gw_daily_sites = gpd.read_file(os.path.join(BASE_PATH, 'groundwater_daily_stations/statons_location'))

NAMES.remove('log.txt')

for NAME in NAMES:
    NHDPlusID_watershed = gpd.read_file(os.path.join(BASE_PATH,fr'SWATplus_by_VPUID/{LEVEL}/{NAME}/SWAT_plus_streams/SWAT_plus_streams.shp')).NHDPlusID.unique()
    num_stations = len(gw_daily_sites[gw_daily_sites.NHDPlusID.isin(NHDPlusID_watershed)])
    if num_stations>0:
        os.makedirs(os.path.join(BASE_PATH,fr'SWATplus_by_VPUID/{LEVEL}/{NAME}/gw_daily_sites'), exist_ok=True)
        print(f'the watershed {NAME} has {num_stations} gw daily observation sites')
        site_files = gw_daily_sites[gw_daily_sites.NHDPlusID.isin(NHDPlusID_watershed)].site_no
        for site_file in site_files:
            try:
                shutil.copy2(os.path.join(BASE_PATH, 'groundwater_daily_stations',f'{site_file}.csv'), os.path.join(BASE_PATH,fr'SWATplus_by_VPUID/{LEVEL}/{NAME}/gw_daily_sites/'))
            except Exception as e:
                print(NAME,e)

the watershed 40500012403 has 1 gw daily observation sites
40500012403 [Errno 2] No such file or directory: '/data/MyDataBase/SWATGenXAppData/groundwater_daily_stations\\420658086082501.csv'
the watershed 40500030307 has 4 gw daily observation sites
the watershed 40500030505 has 1 gw daily observation sites
the watershed 40500040504 has 4 gw daily observation sites
40500040504 [Errno 2] No such file or directory: '/data/MyDataBase/SWATGenXAppData/groundwater_daily_stations\\424300084201001.csv'
the watershed 40601010803 has 1 gw daily observation sites
the watershed 40601020302 has 1 gw daily observation sites
40601020302 [Errno 2] No such file or directory: '/data/MyDataBase/SWATGenXAppData/groundwater_daily_stations\\441503084242201.csv'
the watershed 40601021003 has 3 gw daily observation sites
the watershed 40601050502 has 1 gw daily observation sites
the watershed 40700040102 has 1 gw daily observation sites
40700040102 [Errno 2] No such file or directory: '/data/MyDataBase/SWATGe